In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from titanic_pipeline.config import config
import pandas as pd
import joblib
from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import (
    DataDriftTab,
    DataQualityTab
)
from evidently import ColumnMapping

column_mapping = ColumnMapping(target='survived', prediction='prediction')
model = joblib.load(config.PRODUCTION_MODEL_FILE)

raw_train = pd.read_csv(f"{config.DATASETS_DIR}/data.csv").drop(columns='home.dest')
target_train = raw_train[[config.TARGET]]
raw_train = raw_train.drop(columns=config.TARGET)

dashboard = Dashboard(tabs=[DataQualityTab(), DataDriftTab()])
dashboard.calculate(raw_train, raw_prod, column_mapping=column_mapping)
dashboard.save("../reports/data_quality_drift_raw.html")

/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning: divide by zero encountered in true_divide
  terms = (f_obs_float - f_exp)**2 / f_exp
/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning: divide by zero encountered in true_divide
  terms = (f_obs_float - f_exp)**2 / f_exp
/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning: divide by zero encountered in true_divide
  terms = (f_obs_float - f_exp)**2 / f_exp
/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning: divide by zero encountered in true_divide
  terms = (f_obs_float - f_exp)**2 / f_exp
/Users/andre.bajorat/tec/.titanic/lib/python3.9/site-packages/scipy/stats/_stats_py.py:6766: RuntimeWarning: divide by zero encountered in true_divide
  terms = (f_obs_float - f_exp)**2 / f_exp
/Users/andre.bajorat/tec/.tita

In [2]:
processor = model['preprocessor']
processed_prod = pd.DataFrame(processor.transform(raw_prod.copy()), columns=processor.get_feature_names_out())
processed_train = pd.DataFrame(processor.transform(raw_train.copy()), columns=processor.get_feature_names_out())

processed_prod['prediction'] = model.predict(raw_prod.copy())
processed_train['prediction'] = model.predict(raw_train.copy())

data_drift_report = Dashboard(tabs=[DataQualityTab(), DataDriftTab()])
data_drift_report.calculate(processed_train, processed_prod, column_mapping = column_mapping)
data_drift_report.save("../reports/data_quality_drift_processed.html")

## Release of ground truth data for month January

In [3]:
from evidently.dashboard.tabs import CatTargetDriftTab

target_prod = pd.read_csv(config.DATASETS_DIR + 'prod1_truth.csv')

processed_prod[config.TARGET] = target_prod[config.TARGET]
processed_train[config.TARGET] = target_train[config.TARGET]
data_and_target_drift_report = Dashboard(tabs=[DataDriftTab(), CatTargetDriftTab()])
data_and_target_drift_report.calculate(processed_train, processed_prod, column_mapping = column_mapping)
data_and_target_drift_report.save("../reports/data_n_concept_drift.html")

In [4]:
from evidently.dashboard.tabs import ClassificationPerformanceTab

processed_prod[config.TARGET] = target_prod[config.TARGET]
rf_model = joblib.load(config.BASE_DIR +"/models/RandomForest_Best.sav")
rf_prod = processed_prod.copy()
rf_prod['prediction'] = rf_model.predict(raw_prod.copy())

dashboard = Dashboard(tabs=[ClassificationPerformanceTab(verbose_level=1)])
dashboard.calculate(reference_data=processed_prod, current_data=rf_prod, column_mapping = column_mapping)
dashboard.save('../reports/regression_report.html')

In [7]:
import requests
import json

url = "http://127.0.0.1:8000/prediction"

payload = json.dumps({
  "pclass": 1,
  "name": "Allen, Miss. Elisabeth Walton",
  "sex": "female",
  "age": "29",
  "sibsp": 0,
  "parch": 0,
  "ticket": "24160",
  "fare": "211.3375",
  "cabin": "B5",
  "embarked": "S",
  "boat": "2",
  "body": "?"
})
headers = {
  'Content-Type': 'application/json'
}
print(response.text)


{"prediction":1}


In [16]:
raw_prod = pd.read_csv(config.DATASETS_DIR + 'prod1_data.csv').drop(columns='home.dest')

for record in json.loads(raw_prod.to_json(orient='records')):
    response = requests.request("POST", url, headers=headers, data=json.dumps(record))
    print(response)
    


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [ ]:
new_train_data = raw_prod.copy()
new_train_data[config.TARGET] = target_prod[config.TARGET]
new_train_data.to_csv(config.DATASETS_DIR +"/prod_plus_old_data.csv")